In [ ]:
import numpy as np, torch, dill, time, os
from Ctubes.curve_parametrizations import torus_knot_parameterization_maekawa_scholz
from Ctubes.geometry_utils import rotate_about_axis, regular_polygon, get_bisecting_plane_normals_with_symmetry
from Ctubes.plot_utils import plot_generatrix
from Ctubes.tubes import Directrix, Generatrix, CTube
from Ctubes.target_cross_sections import fix_end_cross_sections 
from Ctubes.opt import CTubeOptimizationProblem
from Ctubes.misc_utils import get_pairings_all
from Ctubes.path_utils import get_name, setup_paths

paths = setup_paths(get_name())

PI = np.pi
TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)
torch.set_printoptions(precision=4)

%load_ext autoreload
%autoreload 2

%matplotlib widget

# Figure 12: 7-fold Torus Unknot

## #1: Original 7-fold Unknot from [Maekawa and Scholz 2024] – Ratio 0.205

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig12_7fold_torus_ms_ratio205")

In [ ]:
# Generatrix: regular N-gon
N = 4
tube_radius = 0.3
generatrix_2d = regular_polygon(N, tube_radius)

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# Directrix: torus knot parametrization from [Maekawa & Scholz 2024]
ratio = 0.205

closed_curve = True
K_per_lobe = 11
n_fold_symm = 7
K = K_per_lobe * n_fold_symm + 1
K_to_M_factor = 6
M = K_to_M_factor * (K - 1) + 1

R = 3
r = ratio * R
ts_disc_curve = torch.linspace(0.0, 2.0 * PI, K)
cps_ref = torus_knot_parameterization_maekawa_scholz(ts_disc_curve, p=1, q=n_fold_symm, r=r, R=R)

In [ ]:
# Define symmetry transform
z_axis = torch.tensor([0.0, 0.0, 1.0], dtype=TORCH_DTYPE)
symm_7_fold = [
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(2*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(4*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(6*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(8*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(10*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(12*PI/7)),
]
symmetry_transforms = symm_7_fold

# Select only one part of the curve
K = (K - 1) // n_fold_symm + 1 #+ 15
M = (M - 1) // n_fold_symm + 1 #+ 30
cps_ref = cps_ref[:K]

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M, symmetry_transforms=symmetry_transforms)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

plane_normals = get_bisecting_plane_normals_with_symmetry(directrix)

tube = CTube(
    directrix, generatrix, plane_normals,
    symmetry_transforms=symmetry_transforms
)

In [ ]:
# Pickle C-Tube
pkl_file = os.path.join(paths["output_opt"], "tube.pkl")
dill.dump(tube, open(pkl_file, "wb"))

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

In [ ]:
# Save initial state
opt_prob = CTubeOptimizationProblem(tube, opt_weights={}, objective_args={})
opt_prob.save_meshes(paths=paths)
opt_prob.save_optimization_results(paths=paths)  # initial state

## #2: Original 7-fold Unknot from [Maekawa and Scholz 2024] – Ratio 0.200

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig12_7fold_torus_ms_ratio200")

In [ ]:
# Directrix: torus knot parametrization from [Maekawa & Scholz 2024]
ratio = 0.200

closed_curve = True
K_per_lobe = 11
n_fold_symm = 7
K = K_per_lobe * n_fold_symm + 1
K_to_M_factor = 6
M = K_to_M_factor * (K - 1) + 1

R = 3
r = ratio * R
ts_disc_curve = torch.linspace(0.0, 2.0 * PI, K)
cps_ref = torus_knot_parameterization_maekawa_scholz(ts_disc_curve, p=1, q=n_fold_symm, r=r, R=R)

In [ ]:
# Define symmetry transform
z_axis = torch.tensor([0.0, 0.0, 1.0], dtype=TORCH_DTYPE)
symm_7_fold = [
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(2*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(4*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(6*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(8*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(10*PI/7)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(12*PI/7)),
]
symmetry_transforms = symm_7_fold

# Select only one part of the curve
K = (K - 1) // n_fold_symm + 1
M = (M - 1) // n_fold_symm + 1
cps_ref = cps_ref[:K]

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M, symmetry_transforms=symmetry_transforms)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)

plane_normals = get_bisecting_plane_normals_with_symmetry(directrix)

tube = CTube(
    directrix, generatrix, plane_normals,
    symmetry_transforms=symmetry_transforms
)

In [ ]:
# Pickle C-Tube
pkl_file = os.path.join(paths["output_opt"], "tube.pkl")
dill.dump(tube, open(pkl_file, "wb"))

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

In [ ]:
# Save initial state
opt_prob = CTubeOptimizationProblem(tube, opt_weights={}, objective_args={})
opt_prob.save_meshes(paths=paths)
opt_prob.save_optimization_results(paths=paths)  # initial state

## #3: C-Tube 7-fold Unknot – Ratio 0.205

In [ ]:
ratio = 0.205
paths_ms = setup_paths(get_name(), test_name="fig12_7fold_torus_ms_ratio{}".format(int(ratio * 1000)))

pkl_file = os.path.join(paths_ms["output_opt"], "tube.pkl")
tube = dill.load(open(pkl_file, "rb"))

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig12_7fold_torus_ours_ratio{}".format(int(ratio * 1000)))

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Target cross-sections
target_cross_sections = fix_end_cross_sections(tube, get_pairings_all(N))

# As second target cross-section, rotate the first cross-section by the first symmetry transform
first_cs = target_cross_sections.target_points[0, ...]
target_cross_sections.target_points[1, ...] = symmetry_transforms[0](first_cs)

In [ ]:
# Set up optimization problem

opt_weights = {
    'match_target_cross_sections': 1e4 / generatrix.aabb_diagonal_length() ** 2,
    'join_ends': 1.0 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'target_cross_sections': target_cross_sections,
    'join_ends_pairings': get_pairings_all(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(False)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(True)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()

## #4: C-Tube 7-fold Unknot – Ratio 0.200

In [ ]:
ratio = 0.200
paths_ms = setup_paths(get_name(), test_name="fig12_7fold_torus_ms_ratio{}".format(int(ratio * 1000)))

pkl_file = os.path.join(paths_ms["output_opt"], "tube.pkl")
tube = dill.load(open(pkl_file, "rb"))

In [ ]:
# Define a path to output specific to the current test case
paths = setup_paths(get_name(), test_name="fig12_7fold_torus_ours_ratio{}".format(int(ratio * 1000)))

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Target cross-sections
target_cross_sections = fix_end_cross_sections(tube, get_pairings_all(N))

# As second target cross-section, rotate the first cross-section by the first symmetry transform
first_cs = target_cross_sections.target_points[0, ...]
target_cross_sections.target_points[1, ...] = symmetry_transforms[0](first_cs)

In [ ]:
# Set up optimization problem

opt_weights = {
    'match_target_cross_sections': 1e4 / generatrix.aabb_diagonal_length() ** 2,
    'join_ends': 1.0 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'target_cross_sections': target_cross_sections,
    'join_ends_pairings': get_pairings_all(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(False)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(True)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []  # no fixed DOF by default

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()